# 第一章：基本提示结构

- [课程](#lesson)
- [练习](#exercises)
- [示例演练场](#example-playground)

## 设置

运行以下设置单元格以加载您的API密钥并建立 `get_completion` 辅助函数。

In [ ]:
%pip install anthropic

# 导入Python内置的正则表达式库。
import re
import anthropic

# 从IPython存储中检索 API_KEY 和 MODEL_NAME 变量。
%store -r API_KEY
%store -r MODEL_NAME

client = anthropic.Anthropic(api_key=API_KEY)

def get_completion(prompt: str, system_prompt=""):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=[
          {"role": "user", "content": prompt}
        ]
    )
    return message.content[0].text

---

## 课程

Anthropic提供两种API：旧版的[文本补全API（Text Completions API）](https://docs.anthropic.com/claude/reference/complete_post)和当前的[消息API（Messages API）](https://docs.anthropic.com/claude/reference/messages_post)。本教程中，我们将专门使用消息API。

使用消息API调用Claude至少需要以下参数：
- `model`: 您打算调用的模型的[API模型名称](https://docs.anthropic.com/claude/docs/models-overview#model-recommendations)。

- `max_tokens`: 在停止生成之前要生成的最大token数量。请注意，Claude可能会在达到此最大值之前停止。此参数仅指定要生成的绝对最大token数量。此外，这是一个*硬性*停止，这意味着它可能导致Claude在生成单词或句子中途停止。

- `messages`: 一个输入消息数组。我们的模型经过训练，可以在交替的 `user`（用户）和 `assistant`（助手）对话轮次中运行。创建新的 `Message` 时，您可以通过 `messages` 参数指定之前的对话轮次，然后模型会生成对话中的下一个 `Message`。
  - 每个输入消息必须是一个包含 `role`（角色）和 `content`（内容）的对象。您可以指定一个 `user` 角色的消息，或者包含多个 `user` 和 `assistant` 消息（如果包含多个，它们必须交替出现）。第一个消息必须始终使用 `user` 角色。

还有一些可选参数，例如：
- `system`: 系统提示——更多内容将在下面讨论。

- `temperature`: Claude响应的可变性程度。对于这些课程和练习，我们将 `temperature` 设置为0。

有关所有API参数的完整列表，请访问我们的[API文档](https://docs.anthropic.com/claude/reference/messages_post)。

### 示例

让我们看看Claude如何响应一些格式正确的提示。对于以下每个单元格，运行该单元格（`shift+enter`），Claude的响应将出现在代码块下方。

In [ ]:
# 提示
PROMPT = "Hi Claude, how are you?"

# 打印 Claude 的响应
print(get_completion(PROMPT))

In [ ]:
# 提示
PROMPT = "你能告诉我海洋的颜色吗?"

# 打印 Claude 的响应
print(get_completion(PROMPT))

In [ ]:
# 提示
PROMPT = "席琳·迪翁是哪一年出生的？"

# 打印 Claude 的响应
print(get_completion(PROMPT))

现在，让我们看看一些不包含正确Messages API格式的提示。对于这些格式错误的提示，Messages API会返回一个错误。

首先，我们有一个Messages API调用示例，其中 `messages` 数组缺少 `role` 和 `content` 字段。

In [ ]:
# 获取 Claude 的响应
response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"Hi Claude, how are you?"}
        ]
    )

# 打印 Claude 的响应
print(response[0].text)

这是一个未能交替使用 `user` 和 `assistant` 角色的提示。

In [ ]:
# 获取 Claude 的响应
response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"role": "user", "content": "席琳·迪翁是哪一年出生的？"},
          {"role": "user", "content": "而且, 你能告诉我一些关于她的其他事实吗？"}
        ]
    )

# 打印 Claude 的响应
print(response[0].text)

`user` 和 `assistant` 消息**必须交替出现**，并且消息**必须以 `user` 轮次开始**。您可以在提示中包含多个 `user` 和 `assistant` 对（就像模拟多轮对话一样）。您也可以在末尾的 `assistant` 消息中填入一些词语，让Claude从您中断的地方继续（这将在后续章节中详细介绍）。

#### 系统提示

您还可以使用**系统提示**。系统提示是一种在“用户”轮次中向Claude提出问题或任务之前，**为其提供上下文、指令和指导**的方式。

从结构上讲，系统提示独立于 `user` 和 `assistant` 消息列表，因此属于一个单独的 `system` 参数（请查看Notebook [设置](#setup) 部分中 `get_completion` 辅助函数的结构）。

在本教程中，凡是可能用到系统提示的地方，我们都在您的补全函数中为您提供了 `system` 字段。如果您不想使用系统提示，只需将 `SYSTEM_PROMPT` 变量设置为空字符串即可。

#### 系统提示示例

In [ ]:
# 系统提示
SYSTEM_PROMPT = "你的回答应该总是一系列批判性思维问题，以推动对话（不要提供问题的答案）。不要实际回答用户的问题。"

# 提示
PROMPT = "天空为什么是蓝的？"

# 打印 Claude 的响应
print(get_completion(PROMPT, SYSTEM_PROMPT))

为什么要使用系统提示？一个**精心编写的系统提示可以从多方面提升Claude的性能**，例如增强Claude遵循规则和指令的能力。欲了解更多信息，请访问我们关于[如何与Claude一起使用系统提示](https://docs.anthropic.com/claude/docs/how-to-use-system-prompts)的文档。

现在我们将深入一些练习。如果您想在不更改上方任何内容的情况下，尝试课程中的提示，请一直滚动到课程Notebook的底部，访问[**示例演练场**](#example-playground)。

---

## 练习
- [练习 1.1 - 数到三](#exercise-11---counting-to-three)
- [练习 1.2 - 系统提示](#exercise-12---system-prompt)

### 练习 1.1 - 数到三
使用正确的 `user` / `assistant` 格式，编辑下面的 `PROMPT`，让Claude**数到三**。输出还将指示您的解决方案是否正确。

In [ ]:
# 提示 - 这是您唯一需要更改的字段
PROMPT = "[替换此文本]"

# 获取Claude的响应
response = get_completion(PROMPT)

# 评估练习正确性的函数
def grade_exercise(text):
    pattern = re.compile(r'^(?=.*1)(?=.*2)(?=.*3).*$', re.DOTALL)
    return bool(pattern.match(text))

# 打印Claude的响应和相应的评分
print(response)
print("\n--------------------------- 评分 ---------------------------")
print("此练习已正确解决：", grade_exercise(response))

❓ 如果您想要提示，请运行下面的单元格！

In [ ]:
from hints import exercise_1_1_hint; print(exercise_1_1_hint)

### 练习 1.2 - 系统提示

修改 `SYSTEM_PROMPT`，让Claude像一个3岁小孩一样回应。

In [ ]:
# 系统提示 - 这是您唯一需要更改的字段
SYSTEM_PROMPT = "[替换此文本]"

# 提示
PROMPT = "天空有多大？"

# 获取Claude的响应
response = get_completion(PROMPT, SYSTEM_PROMPT)

# 评估练习正确性的函数
def grade_exercise(text):
    return bool(re.search(r"giggles", text) or re.search(r"soo", text))

# 打印Claude的响应和相应的评分
print(response)
print("\n--------------------------- 评分 ---------------------------")
print("此练习已正确解决：", grade_exercise(response))

❓ 如果您想要提示，请运行下面的单元格！

In [ ]:
from hints import exercise_1_2_hint; print(exercise_1_2_hint)

### 恭喜！

如果您已经解决了到目前为止的所有练习，那么您已准备好进入下一章。祝您提示愉快！

---

# Chapter 1: Basic Prompt Structure

- [Lesson](#lesson)
- [Exercises](#exercises)
- [Example Playground](#example-playground)

## Setup

Run the following setup cell to load your API key and establish the `get_completion` helper function.

In [ ]:
%pip install anthropic

# Import python's built-in regular expression library
import re
import anthropic

# Retrieve the API_KEY & MODEL_NAME variables from the IPython store
%store -r API_KEY
%store -r MODEL_NAME

client = anthropic.Anthropic(api_key=API_KEY)

def get_completion(prompt: str, system_prompt=""):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=[
          {"role": "user", "content": prompt}
        ]
    )
    return message.content[0].text

---

## Lesson

Anthropic offers two APIs, the legacy [Text Completions API](https://docs.anthropic.com/claude/reference/complete_post) and the current [Messages API](https://docs.anthropic.com/claude/reference/messages_post). For this tutorial, we will be exclusively using the Messages API.

At minimum, a call to Claude using the Messages API requires the following parameters:
- `model`: the [API model name](https://docs.anthropic.com/claude/docs/models-overview#model-recommendations) of the model that you intend to call

- `max_tokens`: the maximum number of tokens to generate before stopping. Note that Claude may stop before reaching this maximum. This parameter only specifies the absolute maximum number of tokens to generate. Furthermore, this is a *hard* stop, meaning that it may cause Claude to stop generating mid-word or mid-sentence.

- `messages`: an array of input messages. Our models are trained to operate on alternating `user` and `assistant` conversational turns. When creating a new `Message`, you specify the prior conversational turns with the messages parameter, and the model then generates the next `Message` in the conversation.
  - Each input message must be an object with a `role` and `content`. You can specify a single `user`-role message, or you can include multiple `user` and `assistant` messages (they must alternate, if so). The first message must always use the `user` role.

There are also optional parameters, such as:
- `system`: the system prompt - more on this below.
  
- `temperature`: the degree of variability in Claude's response. For these lessons and exercises, we have set `temperature` to 0.

For a complete list of all API parameters, visit our [API documentation](https://docs.anthropic.com/claude/reference/messages_post).

### Examples

Let's take a look at how Claude responds to some correctly-formatted prompts. For each of the following cells, run the cell (`shift+enter`), and Claude's response will appear below the block.

In [ ]:
# Prompt
PROMPT = "Hi Claude, how are you?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "Can you tell me the color of the ocean?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "What year was Celine Dion born in?"

# Print Claude's response
print(get_completion(PROMPT))

Now let's take a look at some prompts that do not include the correct Messages API formatting. For these malformatted prompts, the Messages API returns an error.

First, we have an example of a Messages API call that lacks `role` and `content` fields in the `messages` array.

In [ ]:
# Get Claude's response
response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"Hi Claude, how are you?"}
        ]
    )

# Print Claude's response
print(response[0].text)

Here's a prompt that fails to alternate between the `user` and `assistant` roles.

In [ ]:
# Get Claude's response
response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"role": "user", "content": "What year was Celine Dion born in?"},
          {"role": "user", "content": "Also, can you tell me some other facts about her?"}
        ]
    )

# Print Claude's response
print(response[0].text)

`user` and `assistant` messages **MUST alternate**, and messages **MUST start with a `user` turn**. You can have multiple `user` & `assistant` pairs in a prompt (as if simulating a multi-turn conversation). You can also put words into a terminal `assistant` message for Claude to continue from where you left off (more on that in later chapters).

#### System Prompts

You can also use **system prompts**. A system prompt is a way to **provide context, instructions, and guidelines to Claude** before presenting it with a question or task in the "User" turn. 

Structurally, system prompts exist separately from the list of `user` & `assistant` messages, and thus belong in a separate `system` parameter (take a look at the structure of the `get_completion` helper function in the [Setup](#setup) section of the notebook). 

Within this tutorial, wherever we might utilize a system prompt, we have provided you a `system` field in your completions function. Should you not want to use a system prompt, simply set the `SYSTEM_PROMPT` variable to an empty string.

#### System Prompt Example

In [ ]:
# System prompt
SYSTEM_PROMPT = "Your answer should always be a series of critical thinking questions that further the conversation (do not provide answers to your questions). Do not actually answer the user question."

# Prompt
PROMPT = "Why is the sky blue?"

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

Why use a system prompt? A **well-written system prompt can improve Claude's performance** in a variety of ways, such as increasing Claude's ability to follow rules and instructions. For more information, visit our documentation on [how to use system prompts](https://docs.anthropic.com/claude/docs/how-to-use-system-prompts) with Claude.

Now we'll dive into some exercises. If you would like to experiment with the lesson prompts without changing any content above, scroll all the way to the bottom of the lesson notebook to visit the [**Example Playground**](#example-playground).

---

## Exercises
- [Exercise 1.1 - Counting to Three](#exercise-11---counting-to-three)
- [Exercise 1.2 - System Prompt](#exercise-12---system-prompt)

### Exercise 1.1 - Counting to Three
Using proper `user` / `assistant` formatting, edit the `PROMPT` below to get Claude to **count to three.** The output will also indicate whether your solution is correct.

In [ ]:
# Prompt - this is the only field you should change
PROMPT = "[Replace this text]"

# Get Claude's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    pattern = re.compile(r'^(?=.*1)(?=.*2)(?=.*3).*$', re.DOTALL)
    return bool(pattern.match(text))

# Print Claude's response and the corresponding grade
print(response)
print("\n--------------------------- GRADING ---------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ If you want a hint, run the cell below!

In [ ]:
from hints import exercise_1_1_hint; print(exercise_1_1_hint)

### Exercise 1.2 - System Prompt

Modify the `SYSTEM_PROMPT` to make Claude respond like it's a 3 year old child.

In [ ]:
# System prompt - this is the only field you should change
SYSTEM_PROMPT = "[Replace this text]"

# Prompt
PROMPT = "How big is the sky?"

# Get Claude's response
response = get_completion(PROMPT, SYSTEM_PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search(r"giggles", text) or re.search(r"soo", text))

# Print Claude's response and the corresponding grade
print(response)
print("\n--------------------------- GRADING ---------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ If you want a hint, run the cell below!

In [ ]:
from hints import exercise_1_2_hint; print(exercise_1_2_hint)

### Congrats!

If you've solved all exercises up until this point, you're ready to move to the next chapter. Happy prompting!

---

## Example Playground

This is an area for you to experiment freely with the prompt examples shown in this lesson and tweak prompts to see how it may affect Claude's responses.

In [ ]:
# Prompt
PROMPT = "Hi Claude, how are you?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "Can you tell me the color of the ocean?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "What year was Celine Dion born in?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Get Claude's response
response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"Hi Claude, how are you?"}
        ]
    )

# Print Claude's response
print(response[0].text)

In [ ]:
# Get Claude's response
response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"role": "user", "content": "What year was Celine Dion born in?"},
          {"role": "user", "content": "Also, can you tell me some other facts about her?"}
        ]
    )

# Print Claude's response
print(response[0].text)

In [ ]:
# System prompt
SYSTEM_PROMPT = "Your answer should always be a series of critical thinking questions that further the conversation (do not provide answers to your questions). Do not actually answer the user question."

# Prompt
PROMPT = "Why is the sky blue?"

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))